In [1]:
import sys
assert sys.version_info >= (3, 5)

import re,os,csv
import numpy as np
import pandas as pd
from unidecode import unidecode
import dedupe

C:\Users\shitiza\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\shitiza\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\shitiza\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


#### Pre-Prcoess & Clean-up Data Sets before entity Matching

In [70]:
def clean_objCols(dfx):
    for column in dfx.columns.to_list() :
        if np.dtype('O') == dfx[column].dtype:
            #print(f'INFO: clean-up col : {column}')
            dfx[column] = (dfx[column]
                .str.replace(r'\[|\]-|,|;|;|"|\'','', regex=True,flags=re.IGNORECASE)
                .str.replace(r'\s+|/',' ', regex=True,flags=re.IGNORECASE)
                .str.strip()
                .str.strip('"')
                .str.strip("'")
                .str.strip('\[')
                .str.strip('\]')
                .str.lower()
                .str.strip()
                )
    return dfx

def cleanup_cols(dfx,outF,maxL=1000):
    return (
    clean_objCols(dfx)
    #.assign(id = lambda df : df.id.astype(int))
    .assign(id = lambda df : df.index)
    .assign(fileName  = lambda df : [f + '__' + str(i) for f,i in zip(df.fileName,df.index)])
    .loc[:maxL,['fileName','id','Title','Keywords']]
    .set_index('fileName')
    .fillna('')
    .to_csv(outF)
    )

#### Read left & right datasets

In [122]:
DATA_DIR = 'CONSOLIDATED_FOR_ENTITY_MATCHING'
   
OUT_DIR = 'ENTITY_MATCH_OUTPUT/pubMED_vs_clinicTrial'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)


o_left_file = DATA_DIR + '/all_pubmed_keyWords_FINAL.csv'
o_right_file = DATA_DIR + '/all_clinictrial_keyWords_FINAL.csv'

left_file = OUT_DIR + '/pubMedF.csv'
right_file = OUT_DIR + '/clinicTrialF.csv'
cleanup_cols(pd.read_csv(o_left_file).assign(fileName = 'pubMedF').astype(str),left_file)
cleanup_cols(pd.read_csv(o_right_file).assign(fileName = 'clinicTrialF').astype(str),right_file)

In [72]:
pubMed_df = pd.read_csv(left_file).set_index('fileName') 
pubMed_df.head(4)

,id,Title,Keywords
fileName,,,
pubmedf__0,0,micra® leadless pacemaker,leadless presenting approach pacemaker datum ®...
pubmedf__1,1,pacemaker complications and costs: a nationwid...,leadless study novel may system datum pmsi sys...
pubmedf__2,2,runaway pacemaker,examination datum implantation revealed excell...
pubmedf__3,3,pacemaker reprogramming rarely needed after de...,action depending needed involved subsequent 17...


In [73]:
clinic_df = pd.read_csv(right_file).set_index('fileName')
clinic_df.head(4)

,id,Title,Keywords
fileName,,,
clinictrialf__0,0,taiwan registry for leadless pacemaker,leadless intervention pacemaker cardiac patien...
clinictrialf__1,1,placement of cardiac pacemaker trial (pocket)-rct,defibrillator pacemaker 691 totally child ). v...
clinictrialf__2,2,cost-utility analysis of ambulatory care compa...,less hours old living 24 aortic patient arm ou...
clinictrialf__3,3,morbidity mortality and gender differences in ...,intervention pacemaker system get criteriather...


In [74]:
data_1 = pubMed_df.to_dict('index')
data_2 = clinic_df.to_dict('index')

In [75]:
data_1['pubmedf__0']


{'id': 0,
 'Title': 'micra® leadless pacemaker',
 'Keywords': 'leadless presenting approach pacemaker datum ® atrial similar traditional pacemakercardiac method therapy technology without miniature expectancy risk providesmicra invasive less high conclusion conventional transvenous crt versus year fibrillation therefore micra patients venous pacemakers device life'}

In [76]:
data_2['clinictrialf__0']

{'id': 0,
 'Title': 'taiwan registry for leadless pacemaker',
 'Keywords': 'leadless intervention pacemaker cardiac patient patients 1therapy received taiwan child efficiency safety pacemaker exclusion procedure pilot 2 registrytaiwan specifically taiwan receive population selection tavi n attempt mri benefit surgical implantation world performance pacemakersuccessfully inclusion criteria pacemakers early registry'}

In [108]:
output_file = OUT_DIR + '/data_matching_output.csv'
settings_file = OUT_DIR + '/data_matching_learned_settings'
training_file = OUT_DIR + '/data_matching_training_f.json'
def descriptions():
        for dataset in (data_1, data_2):
            for record in dataset.values():
                yield record['Title']   
def brands():
        for dataset in (data_1, data_2):
            for record in dataset.values():
                yield record['Keywords']   

In [109]:
    if os.path.exists(settings_file):
        print('reading from', settings_file)
        with open(settings_file, 'rb') as sf:
            linker = dedupe.StaticRecordLink(sf)

    else:
        # Define the fields the linker will pay attention to
        #
        # Notice how we are telling the linker to use a custom field comparator
        # for the 'price' field.
        fields = [
            {'field': 'Title', 'type': 'String'},
            {'field': 'Title', 'type': 'Text', 'corpus': descriptions()},
            {'field': 'Keywords', 'type': 'Text',
             'has missing': True, 'corpus': brands()}
            ]

        # Create a new linker object and pass our data model to it.
        linker = dedupe.RecordLink(fields)

        # If we have training data saved from a previous run of linker,
        # look for it an load it in.
        # __Note:__ if you want to train from scratch, delete the training_file
        if os.path.exists(training_file):
            print('reading labeled examples from ', training_file)
            with open(training_file) as tf:
                linker.prepare_training(data_1,
                                        data_2,
                                        training_file=tf,
                                        sample_size=15000)
        else:
            linker.prepare_training(data_1, data_2, sample_size=15000)

        # ## Active learning
        # Dedupe will find the next pair of records
        # it is least certain about and ask you to label them as matches
        # or not.
        # use 'y', 'n' and 'u' keys to flag duplicates
        # press 'f' when you are finished
        print('starting active labeling...')

        dedupe.console_label(linker)

        linker.train()

        # When finished, save our training away to disk
        with open(training_file, 'w') as tf:
            linker.write_training(tf)

        # Save our weights and predicates to disk.  If the settings file
        # exists, we will skip all the training and learning next time we run
        # this file.
        with open(settings_file, 'wb') as sf:
            linker.write_settings(sf)


reading from ENTITY_MATCH_OUTPUT/data_matching_learned_settings


INFO:dedupe.api:Predicate set:
INFO:dedupe.api:(SimplePredicate: (sameSevenCharStartPredicate, Title), TfidfNGramSearchPredicate: (0.6, Title), TfidfTextSearchPredicate: (0.2, Title))
INFO:dedupe.api:(SimplePredicate: (commonThreeTokens, Title), TfidfTextSearchPredicate: (0.2, Title), TfidfNGramSearchPredicate: (0.6, Title))
INFO:dedupe.api:(SimplePredicate: (firstTwoTokensPredicate, Title), SimplePredicate: (suffixArray, Title), TfidfTextSearchPredicate: (0.2, Title))
INFO:dedupe.api:SimplePredicate: (commonTwoTokens, Keywords)


In [110]:
    # ## Blocking

    # ## Clustering

    # Find the threshold that will maximize a weighted average of our
    # precision and recall.  When we set the recall weight to 2, we are
    # saying we care twice as much about recall as we do precision.
    #
    # If we had more data, we would not pass in all the blocked data into
    # this function but a representative sample.

    print('clustering...')
    linked_records = linker.join(data_1, data_2, 0.0,'one-to-one')

    print('# duplicate sets', len(linked_records))
    # ## Writing Results

    # Write our original data back out to a CSV with a new column called
    # 'Cluster ID' which indicates which records refer to each other.

    cluster_membership = {}
    for cluster_id, (cluster, score) in enumerate(linked_records):
        for record_id in cluster:
            cluster_membership[record_id] = {'Cluster ID': cluster_id,
                                             'Link Score': score}

    with open(output_file, 'w', encoding='utf8') as f:

        header_unwritten = True

        for fileno, filename in enumerate((left_file, right_file)):
            with open(filename, 'r', encoding="utf8") as f_input:
                reader = csv.DictReader(f_input)

                if header_unwritten:

                    fieldnames = (['Cluster ID', 'Link Score', 'source file'] +
                                  reader.fieldnames)

                    writer = csv.DictWriter(f, fieldnames=fieldnames)
                    writer.writeheader()

                    header_unwritten = False

                for row_id, row in enumerate(reader):

                    record_id = str(re.sub(OUT_DIR,'',re.sub('\.\/|F\.csv','',os.path.basename(filename).lower()))).replace('.csv','') + '__' + str(row_id)
                    #print(record_id)
                              
                    cluster_details = cluster_membership.get(record_id, {})
                    row['source file'] = fileno
                    row.update(cluster_details)
                   

                    writer.writerow(row)

clustering...
# duplicate sets 906


#### Sanity check to ensure Cluster-IN & Score in final matched output csv

In [111]:
chk_df = pd.read_csv(output_file)
len(chk_df['Cluster ID'].unique())
g_m = chk_df['Cluster ID'] != np.nan
chk_df[g_m].describe()

,Cluster ID,Link Score,source file,id
count,1812.000000,1812.000000,2002.000000,2002.000000
mean,452.500000,0.284364,0.500000,500.000000
std,261.611711,0.188868,0.500125,289.035861
min,0.000000,0.027700,0.000000,0.000000
25%,226.000000,0.130286,0.000000,250.000000
50%,452.500000,0.233936,0.500000,500.000000
75%,679.000000,0.415492,1.000000,750.000000
max,905.000000,0.928825,1.000000,1000.000000


#### Use 'Link Score' to filter out matches

In [115]:
match_mask = chk_df['Link Score'] >= 0.5
chk_df_filtered = chk_df[match_mask]
chk_df_filtered.to_csv(OUT_DIR + '/data_matching_output_FILTERED_FINAL.csv',index=False)
print(f'Total matches with score > 0.5 : {chk_df_filtered.shape[0]}')

Total matches with score > 0.5 : 284


In [117]:
chk_df_filtered.loc[~chk_df_filtered['Cluster ID'].isna()].sort_values(by=['Cluster ID']).head(6)

,Cluster ID,Link Score,source file,fileName,id,Title,Keywords
890,0.0,0.928825,0,pubmedf__890,890,risk of syncopal recurrences in patients treat...,bradyarrhythmic aetiology bradyarrhythmias tre...
1238,0.0,0.928825,1,clinictrialf__237,237,risk of syncopal relapses in patients treated ...,bradyarrhythmic syncope underwent treated risk...
1933,1.0,0.910573,1,clinictrialf__932,932,tricuspid regurgitation study,higher provide give septum months enroll leads...
736,1.0,0.910573,0,pubmedf__736,736,tricuspid regurgitation and implantable devices,multiphase defibrillator model pacemaker aorti...
574,2.0,0.875985,0,pubmedf__574,574,incidence of pacing-induced cardiomyopathy in ...,frequent underwent pacemaker picm ablation pat...
1237,2.0,0.875985,1,clinictrialf__236,236,risk of pacing-induced cardiomyopathy,valvular may enhanced aortic cardiac conducted...


In [118]:
def agg_count(s):
    return len([v for v in s.unique()])
def agg_list(s):
    return [v for v in s.unique()]

#### Final Matched output with mapped ids Only

In [119]:
dump_df = chk_df_filtered.groupby(by = ['Cluster ID'])['id'].agg([agg_list]).reset_index()
dump_df['pubMed'] = np.array(dump_df.agg_list.to_list())[:,0]
dump_df['clinicTrial'] = np.array(dump_df.agg_list.to_list())[:,1]
dump_df.loc[:,['pubMed','clinicTrial']].to_csv(OUT_DIR + '/data_matching_Project_2_columnsOnly.csv',index=False)


In [120]:
def join_keywords(k1,k2):
    s1 = set(k1.split(' '))
    s2 = set(k2.split(' '))
    return ' '.join(list(s1.intersection(s2)))

merge_CnP_df = dump_df.loc[:,['pubMed','clinicTrial']]
out_merge_CnP_df = (merge_CnP_df
 .merge(pubMed_df,left_on=['pubMed'],right_on=['id']).rename(columns={'id':'pubMed_id','Title':'pubMed_Title','Keywords':'pubMed_keywords'})
 .merge(clinic_df,left_on=['clinicTrial'],right_on=['id']).rename(columns={'id':'clinicTrial_id','Title':'clinicTrial_Title','Keywords':'clinicTrial_keywords'})
 .assign(keywords_common = lambda df : df.apply(lambda x: join_keywords(x.pubMed_keywords, x.clinicTrial_keywords), axis=1) )
 .drop(['pubMed_id','clinicTrial_id'],axis=1)
)

print(f' No common keywords found for {out_merge_CnP_df.keywords_common.isna().sum()} ')
out_merge_CnP_df.to_csv(OUT_DIR + '/data_matching_Project_IMP_columnsOnly.csv')
out_merge_CnP_df.head(5)

 No common keywords found for 0 


,pubMed,clinicTrial,pubMed_Title,pubMed_keywords,clinicTrial_Title,clinicTrial_keywords,keywords_common
0,890,237,risk of syncopal recurrences in patients treat...,bradyarrhythmic aetiology bradyarrhythmias tre...,risk of syncopal relapses in patients treated ...,bradyarrhythmic syncope underwent treated risk...,pacemaker syncope permanent bradyarrhythmic im...
1,736,932,tricuspid regurgitation and implantable devices,multiphase defibrillator model pacemaker aorti...,tricuspid regurgitation study,higher provide give septum months enroll leads...,pacemaker cardiac aortic implantation regurgit...
2,574,236,incidence of pacing-induced cardiomyopathy in ...,frequent underwent pacemaker picm ablation pat...,risk of pacing-induced cardiomyopathy,valvular may enhanced aortic cardiac conducted...,pacemaker year (≥ using %) cardiomyopathy risk...
3,153,1,placement of cardiac pacemaker trial (pocket) ...,system pacemaker cardiac consecutive enroll si...,placement of cardiac pacemaker trial (pocket)-rct,defibrillator pacemaker 691 totally child ). v...,trial pacemaker cardiac rct two
4,894,835,positron emission tomography in infective endo...,pacemaker cardiac lethal cases suspicion 5 val...,positron emission tomography of infection and ...,defibrillator bacteremia inflammatory interven...,pacemaker cardiac emission endocarditis tomogr...


#### Sample output to check matched entitties

In [121]:
dump_dfT=chk_df.groupby(by = ['Cluster ID'])['Title'].agg([agg_list]).reset_index()
np.array(dump_dfT.agg_list.to_list())[:2]

array([['risk of syncopal recurrences in patients treated with permanent pacing for bradyarrhythmic syncope: role of correlation between symptoms and electrocardiogram findings',
        'risk of syncopal relapses in patients treated with permanent pacing for bradyarrhythmic syncope'],
       ['tricuspid regurgitation and implantable devices',
        'tricuspid regurgitation study']], dtype='<U239')

#### We need to create a ground-truth table with few enteries to verify that dedupe is matching as expected
#### Read ground-Truth Table to check prec/recall only. 

In [92]:
gnd_truth_df = pd.read_csv(OUT_DIR + '/gndTable_ProjectF.csv')

In [93]:
out_df = pd.read_csv(OUT_DIR + '/data_matching_Project_2_columnsOnly.csv')

In [94]:
def agg_count(s):
    return len([v for v in s.unique()])
def agg_list(s):
    return [v for v in s.unique()]

def evaluateDuplicates(found_dupes, true_dupes):
    true_positives = found_dupes.intersection(true_dupes)
    false_positives = found_dupes.difference(true_dupes)
    uncovered_dupes = true_dupes.difference(found_dupes)

    print('found duplicate')
    print(len(found_dupes))
    
    print(f'true_positives : {len(true_positives)}, false_positives : {len(false_positives)}, true_dupes : {len(true_dupes)}')

    prec = len(true_positives) / float(len(found_dupes))
    print(f'precision : {prec}')    

    rec = len(true_positives) / float(len(true_dupes))
    print(f'recall : {rec}')
        
    F1 = 2 * (prec * rec) / (prec + rec)
    print(f'F1 score : {F1}')


In [95]:
true_dupes = set(map(tuple, np.array(gnd_truth_df).tolist()))
found_dupes = set(map(tuple, np.array(out_df).tolist()))

In [96]:
evaluateDuplicates(found_dupes,true_dupes)

found duplicate
142
true_positives : 24, false_positives : 118, true_dupes : 24
precision : 0.16901408450704225
recall : 1.0
F1 score : 0.2891566265060241


#### Trick to create ground-truth Table

In [91]:
gnd_mask = chk_df_filtered['Link Score'] >= 0.7
dump_df = chk_df_filtered[gnd_mask].groupby(by = ['Cluster ID'])['id'].agg([agg_list]).reset_index()
dump_df['pubMed'] = np.array(dump_df.agg_list.to_list())[:,0]
dump_df['clinicTrial'] = np.array(dump_df.agg_list.to_list())[:,1]
dump_df.loc[:,['pubMed','clinicTrial']].to_csv(DATA_DIR + '/gndTable_ProjectF.csv',index=False)